In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/FYP/chest_xray_MobileNetV3_OriginalData>.zip -d /content/extracted_files

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras import layers, models, optimizers
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
# Step 1: Set up paths for the dataset
dataset_dir = "/content/extracted_files/chest_xray_MobileNetV3_OriginalData"

# Define paths for training and testing
train_dir = os.path.join(dataset_dir, "/content/extracted_files/chest_xray_MobileNetV3_OriginalData/train")
test_dir = os.path.join(dataset_dir, "/content/extracted_files/chest_xray_MobileNetV3_OriginalData/test")



In [ ]:
# Step 2: Load dataset without augmentation
img_height, img_width = 224, 224  # MobileNetV3 recommended input size


In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="binary",
    subset="training",
)

Found 4187 images belonging to 2 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="binary",
    subset="validation",
)

Found 1045 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode="binary",
    shuffle=False,
)

Found 624 images belonging to 2 classes.


In [ ]:
# Step 3: Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weights_dict)


Class Weights: {0: 1.938425925925926, 1: 0.6738010943031864}


In [ ]:
# Step 4: Define MobileNetV3 model
base_model = MobileNetV3Small(
    input_shape=(img_height, img_width, 3),
    include_top=False,
    weights="imagenet",
)

4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze base model
base_model.trainable = False


In [ ]:
# Add custom layers on top of MobileNetV3
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])



In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Small (Functional)        │ (None, 7, 7, 576)           │         939,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 576)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,013,105 (3.86 MB)

 Trainable params: 73,985 (289.00 KB)

 Non-trainable params: 939,120 (3.58 MB)

In [ ]:
# Step 5: Train the model with class weights
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Adjust as needed
    class_weight=class_weights_dict,
    verbose=1,
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 115s 801ms/step - accuracy: 0.5083 - loss: 0.7154 - val_accuracy: 0.2574 - val_loss: 0.7068
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 142s 791ms/step - accuracy: 0.4603 - loss: 0.6952 - val_accuracy: 0.7426 - val_loss: 0.6660
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 140s 788ms/step - accuracy: 0.6355 - loss: 0.6781 - val_accuracy: 0.2574 - val_loss: 0.6980
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 143s 796ms/step - accuracy: 0.4567 - loss: 0.7033 - val_accuracy: 0.7464 - val_loss: 0.6854
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 139s 763ms/step - accuracy: 0.5554 - loss: 0.6891 - val_accuracy: 0.2584 - val_loss: 0.6994
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 124s 928ms/step - accuracy: 0.4664 - loss: 0.6987 - val_accuracy: 0.7407 - val_loss: 0.6741
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 121s 760ms/step - accuracy: 0.5711 - loss: 0.6979 - val_accuracy: 0.7407 - val_loss: 0.6752
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 125s 943ms/step - accuracy: 0.6087 - loss: 0.68

In [ ]:
# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 13s 616ms/step - accuracy: 0.7453 - loss: 0.6805
Test Accuracy: 70.67%


In [ ]:
# Predictions and classification report
test_generator.reset()
y_pred = (model.predict(test_generator) > 0.5).astype("int32")
y_true = test_generator.classes


20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 597ms/step


In [ ]:
print("\nClassification Report:")
print(classification_report(
    y_true,
    y_pred,
    target_names=test_generator.class_indices.keys(),
    zero_division=0,  # Suppress undefined metric warning
))



Classification Report:
              precision    recall  f1-score   support

      NORMAL       0.58      0.80      0.67       234
   PNEUMONIA       0.85      0.65      0.73       390

    accuracy                           0.71       624
   macro avg       0.71      0.73      0.70       624
weighted avg       0.75      0.71      0.71       624



In [ ]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Confusion Matrix:
[[188  46]
 [137 253]]
